[![Open in colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FangmingXie/collab-workshop-st/blob/main/workbook/exercise2-5.ipynb)

# Exercise 2-5: Typical (and untypical) analyses in spatial transcriptomics

Dataset:
Vizgen Data Release V1.0. May 2021

In [ ]:
# Install and import packages
!pip install umap-learn
from umap import UMAP # this is installed by us

In [ ]:
# Install and import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import time

# set plot style
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set_context('talk')

np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

In [ ]:
# functions (the teacher wrote for you to use later)
def rot2d(x, y, theta, unit='degree'):
    """ rotate data points defined by `x` and `y` by `theta` degree
    """
    a = np.vstack([x,y]).T
    if unit == 'degree':
      theta = theta*np.pi/180 # convert to radian

    R = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    ar = a.dot(R.T)
    return ar[:,0], ar[:,1]

def st_scatter(x, y, gexp=None, vmax_p=98, title='', s=1, cbar_label=''):
    """customized scatter plot -- yesterday's progress
    """
  
    fig, ax = plt.subplots(figsize=(10,8))
    if gexp is not None:
        vmax = np.percentile(gexp, vmax_p)
        g = ax.scatter(x, y, c=gexp, s=s, edgecolor='none', vmax=vmax)
        fig.colorbar(g, label=cbar_label, shrink=0.3)
    else:
        g = ax.scatter(x, y, s=s, edgecolor='none')
  
    ax.set_title(title)
    ax.set_aspect('equal')
  
    return 

In [ ]:
np.random.seed(0) # for reproducibility

## Load data

In [ ]:
f = '/content/vizgen_mouse_brain_S2R2.csv.gz'
data = pd.read_csv(f, index_col=0)
data

## Warm up exercise: Check cell locations, rotation, and visualization of gene expression.

In [ ]:
x = data['x'].values
y = data['y'].values
st_scatter(x, y)

In [ ]:
theta = 40
xr, yr = rot2d(x, y, theta)
st_scatter(xr, yr)

In [ ]:
gene_names = [
  'Slc17a7',
  'Slc17a6',
  'Gad1',
  'Gfap',
  'Olig1',
  'Drd1',
  'Drd2',
]

for gene_name in gene_names:
  # modify code here to plot gene expression pattern for 
  # all the genes on the above list
  pass

## Exercise 1: Dimensionality reduction using PCA

In [ ]:
ftrs = data.iloc[:,2:]
ftrs

In [ ]:
%%time
# do PCA here on the feature matrix (483 genes and ~80k cells)
# check this doc:
# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

num_pc = 20

# pca_obj = PCA(...)
# pcs = pca_obj.fit_transform(...)
# print(pcs.shape)

In [ ]:
# make a plot of the Number of PCs vs their explained variance ratio
# learn from the link below: 
# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
# to see how to get the explained variance ratio for each PC and plot it

# plt.plot(1+np.arange(num_pc), , 'o')
plt.xlabel('Number of PCs')
plt.ylabel('Variance explained')

In [ ]:
# make a plot of the Number of PCs vs their cumulative explained variance ratio
# you may want to use `np.cumsum` to calculate the cumulative sum of a vector.

# plt.plot(1+np.arange(num_pc), , 'o')

plt.xlabel('Number of PCs')
plt.ylabel('Cumulative variance explained')

In [ ]:
# plot PC1 vs PC2 colored by Slc17a7
pc1 = pcs[:,0]
pc2 = pcs[:,1]
gexp = np.log10(1+ftrs['Slc17a7'])
st_scatter(pc1, pc2, gexp, title="PC 1-2 colored by Slc17a7")

In [ ]:
# Now can you make a plot of PC3 vs PC4 colored by Gad1?
# pc3 = 
# pc4 = 
# gexp = 
st_scatter(pc3, pc4, gexp, title="PC3 vs PC4 colored by Gad1")

## Exercise 2: Dimensionality reduction using UMAP
this takes long. We will use PCs (n=20) as input and reduce it down to UMAP components (n=2)

In [ ]:
pcs = PCA(n_components=20).fit_transform(ftrs.values)
pcs.shape

In [ ]:
%%time
# takes ~3 min 

# VERY similar syntax to how you do PCA
# ucs =  

# print(ucs.shape)

In [ ]:
# complete the code below to plot UMAP coordinates

# uc1 =   ## this is the first UMAP component
# uc2 =   ## this is the second UMAP component
# gexp = np.log10(1+ftrs['Slc17a7'])
st_scatter(uc1, uc2, gexp, title="UMAP coords. colored by Slc17a7")

## Exercise 3: Normalization
- Neither PCA nor UMAP are magic, in the sense that they do not separate technical noise from biological signals.
- Therefore we often need to do normalization on raw data to remove known technical artifacts.
- In single-cell transcriptomics, a rule of thumb is to normalize raw counts by cell size (or cell library size) and do log transformation. (why?)

In [ ]:
# normalization by size (total counts or volume)
cov = ftrs.sum(axis=1)
medcov = np.median(cov)

# normalization by log(1+x) (squashes different orders of magnitude together)
ftrs_normed = ftrs.divide(cov/medcov, axis=0)
ftrs_normed = np.log10(1+ftrs_normed)
ftrs_normed

In [ ]:
# can you check if the normalization does its job? # how?




In [ ]:
# repeat PCA and UMAP like above using the normalized features (ftrs_normed) 

# this may take a while
ti = time.time()

print("PCA first...", end='')
### first do pca (20 PCs)
# pcs_normed = 
print(pcs_normed.shape)

print("UMAP second...", end='')
### then send the PC results to UMAP to further reduce dimensions to 2 components
# ucs_normed = 

print(f"done in {time.time()-ti:.2f}s")
print(ucs_normed.shape)

In [ ]:
# plot the new UMAP coordinates
uc1 = ucs_normed[:,0]
uc2 = ucs_normed[:,1]
gexp = np.log10(1+ftrs['Slc17a7'])
st_scatter(uc1, uc2, gexp, title="UMAP coords. colored by Slc17a7")


### visualize individual genes in XY and UMAP 
- having both spatial (XY) and UMAP coordinates, we can plot them side-by-side to better reveal what is in our data

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8*2,8))
ax = axs[0]
ax.scatter(xr, yr, s=1, edgecolor='none')
ax.set_title('XY (spatial distribution)')
ax.set_aspect('equal')

ax = axs[1]
ax.scatter(uc1, uc2, s=1, edgecolor='none')
ax.set_title('UMAP (molecular similarity)')
ax.set_aspect('equal')

In [ ]:
def st_scatter_xy_umap(x, y, ux, uy, ftrs_normed, gene=None):
  """plot XY and UMAP side by side colored by gene
  """
  if gene is not None:
      gval = ftrs_normed[gene]

  fig, axs = plt.subplots(1, 2, figsize=(8*2,8))
  fig.suptitle(gene)
  
  ax = axs[0]
  g = ax.scatter(xr, yr, c=gval, s=1, edgecolor='none')
  ax.set_title('XY (spatial distribution)')
  ax.set_aspect('equal')
  
  ax = axs[1]
  ax.scatter(ux, uy, c=gval, s=1, edgecolor='none')
  ax.set_title('UMAP (molecular similarity)')
  ax.set_aspect('equal')
  fig.colorbar(g, ax=ax, shrink=0.3, label='normed counts\n(by size and log10(x+1))')

  return 

In [ ]:
st_scatter_xy_umap(xr, yr, uc1, uc2, ftrs_normed, 'Slc17a7')

In [ ]:
for gene in [
    'Slc17a6', 'Slc17a7', # exc; different sub populations
    'Gad1',  # inh
    'Gfap', 'Olig1', # spatially co-localized; molecularly distinct
    'Drd1', 'Drd2',  # spatially co-localized; molecularly distinct
    ]:
    # modify code below to repeat the plot above for all genes listed here
    pass

## Exercise 4: Clustering - KMeans
- k specifies how many clusters we aim to identify

In [ ]:
# visualize clusters
def plot_cluster(clsts, x, y, ux, uy, s=1):
  """
  """
  from matplotlib import colors

  unq_clsts, inv = np.unique(clsts, return_inverse=True)
  n_unq = len(unq_clsts)
  # colors = np.array(sns.color_palette('husl', n_unq))
  # c_vec = colors[inv]

  cmap = plt.cm.jet
  norm = colors.BoundaryNorm(np.arange(-0.5, n_unq, 1), cmap.N)

  fig, axs = plt.subplots(1, 2, figsize=(8*2,8))
  
  ax = axs[0]
  g = ax.scatter(x, y, norm=norm, cmap=cmap, c=clsts, s=s, edgecolor='none')
  ax.set_title('XY (spatial distribution)')
  ax.set_aspect('equal')
  
  ax = axs[1]
  ax.scatter(ux, uy, norm=norm, cmap=cmap, c=clsts, s=s, edgecolor='none')
  ax.set_title('UMAP (molecular similarity)')
  ax.set_aspect('equal')

  fig.colorbar(g, ax=ax, label='clusters', ticks=np.arange(n_unq))

In [ ]:
%%time
# clustering (k means clustering)
# takes ~1.5min

from sklearn.cluster import KMeans

# clustering -- in the end every cell will be assigned to a cluster
k = 10 # generate 10 clusters 
# ....
# clsts = ... 

In [ ]:
# plot clustering results (assuming `clsts` is a list of cluster labels for each cell)
plot_cluster(clsts, xr, yr, uc1, uc2)

In [ ]:
# Now can you repeat K-Means clustering to get
# k = 3, 5, 10, 20 clusters respectively, and plot their distributions
# in XY and UMAP coordinates?



## Bonus point 1: Cluster centroid (pseduo-bulk) profiles
- computationally merging cells from the same cell type (like a in-silico bulk RNA-seq) 

Steps:
1. create a new dataframe called `tmp` which is a copy of `ftrs_normed`
2. add a new column in `tmp` called `clst` and assign cluster labels of the cells to this column
3. use the powerful tool `tmp.groupby('clst').mean()` to calculate the mean per cluster for every gene.
4. does the result make sense to you?


Visualization:
5. z-score the results (cell-type by gene expression profiles) per gene across cell types. If `X` is your centroid matrix (cell type by gene), its zscore should be `(X-X.mean())/X.std()`. 

6. visualize the above matrix using the heatmap command from the Seaborn package (`sns.heatmap()`)

In [ ]:
# ctrds = ...
# ctrds_zscore = ...

In [ ]:
# visualize the results
sns.clustermap(ctrds_zscore.T, figsize=(6,10), cmap='coolwarm', center=0)

## Bonus point 2: Enrichment analysis of spatial proximity
- in the style of [Fang et al. 2022 Science](https://www.science.org/doi/10.1126/science.abm1741) (Figure 4)

- enrichment analysis (spatial neighbors of clusters) -- 
 which cell type pairs are enriched in spatial proximity

Steps:
- for each cell, get their k nearest spatial neighbors. 
  you may use `from sklearn.neighbors import NearestNeighbors`

- count the results for each cell type pair -- how many connections there are between each pair.
  you may use the function below `get_clst2clst_counts`

- compare it with what's expected by random homogenous cell types....; are they enriched?
  you may want to repeat the steps above after randomly shuffling cell type labels across cells using `np.random.choice(..., replace=False)`

In [ ]:
def get_clst2clst_counts(clsts, knn):
  """Given cluster labels (for each cell) and spatial neighbors (indices; including self), 
  Calculate how many connections (neighbor pairs) each cluster pairs have.
  Return this information as a cluster-by-cluster count matrix
  """
  from scipy import sparse
  knn_clsts = clsts[knn] # cluster label for kNN
  n_unq_clsts = len(np.unique(clsts)) # number of clusters
  
  shape = (n_unq_clsts, n_unq_clsts)
  _self = knn_clsts[:,0]
  _others = knn_clsts[:,1:]
  rows = np.repeat(_self, (k-1)) # repeat 
  cols = np.hstack(_others)
  assert len(rows) == len(cols) 
  data = np.repeat(1, len(rows))
  clst2clst_counts = sparse.coo_matrix((data, (rows, cols)), shape=shape).todense() 
  clst2clst_counts = (clst2clst_counts + clst2clst_counts.T) # each connection counted twice.
  return clst2clst_counts

In [ ]:
# for each cell, get their k nearest spatial neighbors
from sklearn.neighbors import NearestNeighbors
xy = np.vstack([xr, yr]).T

k = 1+6 # 1(self) + 6 nearest neighbors
# knn = 
print(knn.shape)
print(knn)

In [ ]:
# count number of connections for every cell type pair
clst2clst_counts = get_clst2clst_counts(clsts, knn)

In [ ]:

# redo the above counting after shuffing cell type labels 
# clsts_shuff = 
clst2clst_counts_shuff = get_clst2clst_counts(clsts_shuff, knn)

In [ ]:
# visualize the difference between original vs shuffled cell type labels
plot_cluster(clsts, xr, yr, uc1, uc2)
plot_cluster(clsts_shuff, xr, yr, uc1, uc2)

In [ ]:
sns.heatmap(clst2clst_counts)
plt.gca().set_aspect('equal')
plt.title('observed')

In [ ]:
sns.heatmap(clst2clst_counts_shuff)
plt.gca().set_aspect('equal')
plt.title('expected')

In [ ]:
sns.heatmap(np.log2((1+clst2clst_counts)/(1+clst2clst_counts_shuff)), # 1 is pseudo count to avoid 0 in log. 
            cmap='coolwarm',
            vmax=2,vmin=-2,
            )
plt.gca().set_aspect('equal')
plt.title('enriched: log2(observed/expected)')

## Bonus follow-up questions
- How do we make sure the expected pattern is stable (not a fluke of 1 particular random shuffle)? 
- How do we evaluate the statistical significance of the enrichment?

Answer:
- We can repeat random shuffling many many times (100, 1000, 10,000) and recount number of spatial neighbors between cell-type pairs; this will stablize the expected pattern and allow us to estimate emprical p-values.